In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
from pegasus import transformer

In [3]:
vocab_size = 32000
hidden_size = 768
filter_size = 3072
num_encoder_layers = 12
num_decoder_layers = 12
num_heads = 12
label_smoothing = 0.0
dropout = 0.0

In [4]:
model = transformer.TransformerEncoderDecoderModel(vocab_size, hidden_size,
                                                   filter_size, num_heads,
                                                   num_encoder_layers,
                                                   num_decoder_layers,
                                                   label_smoothing, dropout)

In [5]:
X = tf.placeholder(tf.int64, (None, None))
top_p = tf.placeholder(tf.float32, None, name = 'top_p')
temperature = tf.placeholder(tf.float32, None, name = 'temperature')
outputs = model.predict({"inputs": X,}, tf.shape(X)[1], beam_size = 1,
                       top_p = top_p, temperature = temperature)




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor





In [20]:
logits = tf.identity(outputs['outputs'], name = 'logits')
logits

<tf.Tensor 'logits:0' shape=(?, ?) dtype=int64>

In [6]:
import sentencepiece as spm

vocab = '/home/husein/b2b/sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s)
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [7]:
import tensorflow as tf

ckpt_path = tf.train.latest_checkpoint('gs://mesolitica-tpu-general/pegasus-summarization-base')
ckpt_path

'gs://mesolitica-tpu-general/pegasus-summarization-base/model.ckpt-900000'

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [9]:
saver = tf.train.Saver()
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/pegasus-summarization-base/model.ckpt-900000


In [15]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string.replace('\n', ' '))).strip()

In [16]:
string = """
Tidak ada apa-apa mengenai mesyuarat fakulti biologi Jumaat lalu di University of Alabama di Huntsville yang membayangkan pembunuhan yang akan berlaku, Profesor Debra Moriarity memberitahu wartawan Rabu. "Ia sebenarnya adalah satu-satunya mesyuarat fakulti yang sangat santai dan biasa," ahli biokim memberitahu CNN affiliate WAAY mengenai 13 orang yang duduk di sekitar meja bujur di Bilik 369 di Shelby Centre for Science and Technology. "Peristiwa akan datang, kelas penjadualan, belanjawan. Ia sebenarnya merupakan salah satu daripada mesyuarat fakulti yang paling mudah yang kami ada." Antara peserta adalah Amy Bishop, seorang ahli genetik yang terlatih Harvard dengan siapa Moriarity telah membangun hubungan profesional yang dipupuk oleh hakikat bahawa kedua-dua wanita bekerja dengan budaya sel. "Kadang-kadang anda meminjam sesuatu dari satu sama lain," katanya. "Kami telah bercakap mengenai cadangan pemberian cadangan bersama." Selepas kira-kira sejam, sebelum 4hb, Bishop - yang baru-baru ini dinafikan tempohnya - menamatkan tenang. "Secara tiba-tiba, dia hanya berdiri dan ditembak," kata Moriarity. Moriarity bertindak balas dengan cepat, menjatuhkan ke tangan dan lututnya di atas permaidani kelabu. "Hanya jatuh ke lantai dan merangkak di bawah meja dan merangkak ke arah Amy," kata Moriarity, yang memberi tumpuan kepada satu perkara. "Maksud saya, kamu merangkak di bawah meja, kamu melihat kaki orang yang menembak di atas meja, saya merangkul kakinya dan, saya tidak tahu apa yang saya fikirkan, saya tidak memikirkan apa-apa. hanya berfikir: \'Ambil dia!\' "Dan dia menghalang saya. Maksud saya, dia menarik kaki kakinya percuma dan saya berada di ambang pintu dengan belakang saya jenisnya. Dan saya fikir dia cuba menembak saya, tetapi ketika saya mula berteriak kepadanya, \'Amy, Amy, berfikir tentang cucu saya, berfikir tentang anak perempuan saya! Ini saya! Saya telah membantu anda sebelum ini; Saya akan membantu anda lagi! Jangan buat Amy ini! Jangan lakukan ini! \'"Uskup kemudian melangkah masuk ke dalam dewan, menunjuk pistol di Moriarity dan menarik pencetus, kata ahli biologi." Ia diklik, dan ia diklik lagi, dan saya merangkak kembali ke dalam bilik dan menutup pintu dan dia ditinggalkan di dalam dewan. "Orang-orang yang terselamat itu beraksi. Satu orang mengunci pintu kayu, satu lagi mengetuk meja ke atasnya, yang lain memindahkan peti sejuk ke tempatnya untuk menghalang pintu, yang lain dipanggil 911, yang lain berpindah ke Tiga orang yang maut, tiga lagi yang cedera, dua daripadanya kekal dirawat di rumah sakit pada hari Rabu dalam keadaan kritikal, menurut seorang jurubicara Hospital Huntsville, yang ketiga dilepaskan, Moriarity, yang menyertai fakulti sekolah pada tahun 1984, mengatakan bahawa mangsa tidak menjejaskan rancangannya untuk kekal di sekolah dan dia menolak apa-apa cadangan bahawa peranannya untuk mendapatkan Bishop di luar bilik itu adalah heroik. "Dia mengikuti saya di dewan dan kemudian senjata itu macet dan saya boleh mendapatkan ba ck di dalam bilik, "kata Moriarity. "Itu bukan seorang pahlawan. Itu hanya Tuhan yang melihat kamu." Dia berkata dia mempunyai sedikit masa untuk berfikir. "Dari awal hingga akhirnya kami mendapat sesuatu yang tidak dapat dibendung, ia tidak boleh melebihi 20 saat," katanya. Moriarity terus menolak cadangan bahawa apa-apa boleh dilakukan untuk melindungi mangsa. "Tidak ada cara untuk menjangka ini," katanya. "Dan tiada apa yang dapat dilakukan untuk menghentikannya, semuanya berlaku terlalu pantas." Dan dia bimbang bahawa apa-apa cubaan untuk mengetatkan keselamatan boleh membawa kesan negatif. "Ada kejahatan di dunia, malangnya orang baik disakiti oleh itu, tetapi universiti adalah tempat pemikiran bebas dan kebebasan untuk meneroka idea-idea dan mencari pengetahuan baru dan anda tidak mahu meletakkan sesuatu di tempat yang meredakannya. " Moriarity kembali ke pejabatnya pada hari Rabu dan berkata dia merancang untuk meneruskan pengajaran minggu depan. Dia meramalkan bahawa, dengan bantuan ubat anti-kecemasan, dia akan dapat tidur malam Rabu. "Saya telah bercakap dengan keluarga dan kawan-kawan dan hanya mendapatkan sokongan mereka membantu anda menanganinya," katanya. "Saya fikir sekarang kebanyakan dari kita mahu kembali ke sana dan mendapatkan sesuatu yang berlaku, membuat rancangan untuk siapa yang akan menutup kelas." Satu perkhidmatan peringatan untuk menghormati kehidupan orang mati - ahli fakulti Maria Davis, Adriel Johnson dan Gopi Podila - akan diadakan pada hari Jumaat.
"""

In [17]:
string2 = """
Gabungan parti Warisan, Pakatan Harapan, dan Upko hari ini mendedahkan calon-calon masing-masing untuk pilihan raya negeri Sabah, tetapi ketika pengumuman itu berlangsung, perwakilan PKR di dewan itu dilihat ‘gelisah’ seperti ‘tidak senang duduk’.

Sekumpulan anggota PKR kemudian dilihat meninggalkan dewan di Pusat Konvensyen Antarabangsa Sabah di Kota Kinabalu selepas berbincang dengan ketua PKR Sabah Christina Liew.

Semakan senarai-senarai calon berkenaan mendapati PKR hanya memperolehi separuh daripada jumlah kerusi yang diharapkan.

Semalam, PKR Sabah mengumumkan akan bertanding di 14 kerusi tetapi ketika Presiden Warisan Shafie Apdal mengumumkan calon gabungan tersebut hari ini, PKR hanya diberikan tujuh kerusi untuk bertanding.

Kerusi yang diberikan adalah Api-Api, Inanam, Tempasuk, Tamparuli, Matunggong, Klias, dan Sook.

Klias dan Sook adalah dua kerusi yang diberikan kepada PKR, sementara lima kerusi selebihnya pernah ditandingi oleh PKR pada pilihan raya umum 2018.

Dalam pengumuman PKR Sabah semalam, parti itu menjangkakan Warisan akan turut menyerahkan kerusi Kemabong, Membakut, dan Petagas kepada mereka.

Walau bagaimanapun, Warisan menyerahkan kerusi Kemabong kepada Upko dan mengekalkan bertanding untuk kerusi Membakut dan Petagas.

PKR juga menuntut empat daripada 13 kerusi baru yang diperkenalkan iaitu Segama, Limbahau, Sungai Manila, dan Pintasan tetapi Warisan membolot semua kerusi itu.

Sebagai pertukaran untuk kerusi yang diintainya, PKR bersedia untuk menyerahkan kerusi Kadaimaian, Kuala Penyu, dan Karanaan. Namun, ini dijangka tidak akan berlaku memandangkan parti tersebut tidak berpuas hati dengan agihan kerusi seperti yang diharapkan itu.

Selepas perwakilan dari PKR dan Liew keluar dari dewan tersebut, wartawan kemudian menyusuri Liew untuk mendapatkan penjelasannya.

Walau bagaimanapun, Liew enggan memberikan sebarang komen dan berkata bahawa dia ingin ke tandas.

Liew dan perwakilan PKR kemudian tidak kembali ke dalam dewan tersebut.

Apabila calon pilihan raya yang diumumkan diminta naik ke atas pentas untuk sesi bergambar, Liew tidak kelihatan.

Bilangan kerusi yang ditandingi oleh PKR kali ini hanya kurang satu kerusi daripada yang ditandingi parti itu pada PRU 2018.

Dalam perkembangan berkaitan, DAP dan Amanah dikatakan tidak mempunyai sebarang masalah dengan kerusi yang diberikan untuk PRN Sabah.

Sementara itu, Presiden Upko Madius Tangau enggan mengulas adakah dia berpuas hati dengan agihan kerusi tersebut. Madius kekal di majlis tersebut sehingga ia berakhir.

Partinya diberikan 12 kerusi, iaitu lebih tujuh kerusi berbanding PRU lalu.

DAP dan Amanah akan bertanding di bawah logo Warisan sementara PKR dan Upko akan menggunakan logo masing-masing.

DAP akan bertanding di tujuh kerusi, jumlah yang sama seperti yang mereka tandingi pada PRU lalu, sementara Amanah diberi satu kerusi.

Warisan akan bertanding sebanyak 54 kerusi.

Perkembangan terbaru ini mungkin mencetuskan pergeseran di antara PKR dan Warisan. PKR boleh memilih untuk bertanding di lebih banyak kerusi daripada 14 yang dituntutnya manakala Warisan juga boleh bertanding di kerusi sekutunya.

Barisan pemimpin tertinggi PKR dan Warisan hanya mempunyai dua hari sebelum hari penamaan calon pada Sabtu untuk mengurangkan pergeseran.
"""

In [18]:
string3 = """
Penubuhan universiti sukan seperti diutarakan Ketua Unit Sukan Kementerian Pengajian Tinggi, Dr Pekan Ramli dan disokong Pakar Pembangunan Sukan dan Reakreasi Luar, Universiti Pendidikan Sultan Idris (UPSI), Prof Dr Md Amin Md Taaf seperti disiarkan akhbar ini, memberikan sinar harapan kepada kewujudan institusi sedemikian.

Ia menjadi impian atlet negara untuk mengejar kejayaan dalam bidang sukan dan kecemerlangan dalam akademik untuk menjamin masa depan lebih baik apabila bersara daripada arena sukan kelak.

Pelbagai pandangan, idea, kaedah, bukti dan cadangan dilontarkan pakar berikutan pentingnya universiti sukan yang akan memberi impak besar sama ada pada peringkat kebangsaan mahupun antarabangsa.

Negara lain sudah lama meraih laba dengan kewujudan universiti sukan seperti China, Korea, Japan, Taiwan, India dan Vietnam. Mereka menghasilkan atlet universiti yang mempamerkan keputusan cemerlang pada peringkat tinggi seperti Sukan Olimpik, Kejohanan Dunia dan Sukan Asia.

Justeru, kejayaan mereka perlu dijadikan rujukan demi memajukan sukan tanah air. Jika kita merujuk pendekatan Asia, kewujudan universiti sukan penting dan memberi kesan positif dalam melonjakkan prestasi sukan lebih optimum.

Namun, jika kita melihat pendekatan Eropah, universiti sukan bukan antara organisasi atau institusi penting yang diberi perhatian dalam menyumbang kepada pemenang pingat.

Antara isu dalam universiti sukan ialah kos tinggi, lokasi, prasarana sukan, pertindihan kursus dengan universiti sedia ada dan impak terhadap dunia sukan negara hingga mengundang persoalan kewajaran dan kerelevanan penubuhannya.

Namun sebagai bekas atlet memanah negara dan Olympian (OLY) di Sukan Olimpik 2004 di Athens, Greece serta bekas pelajar Sekolah Sukan Bukit Jalil hingga berjaya dalam dunia akademik, saya mendapati terdapat beberapa faktor sering menjadi halangan dalam rutin harian mereka.

Antaranya, faktor masa yang terpaksa bergegas menghadiri kuliah selepas tamat sesi latihan yang mengambil masa 15 hingga 20 minit dengan menunggang motosikal; kereta (20-30 minit) atau pengangkutan disediakan Majlis Sukan Negara (MSN) ke Universiti Putra Malaysia (UPM).

Jika mereka menuntut di Universiti Teknologi MARA (UiTM) atau Universiti Malaya (UM), ia mungkin lebih lama.

Walaupun di universiti tersedia dengan kemudahan kolej dan kemudahan sukan, mereka memilih pulang ke MSN untuk menjalani latihan bersama pasukan dan jurulatih di padang atau gelanggang latihan rasmi.

Ini berlanjutan selagi bergelar atlet negara yang perlu memastikan prestasi sentiasa meningkat dari semasa ke semasa tanpa mengabaikan tugas sebagai pelajar.

Alangkah baiknya jika sebahagian Sekolah Sukan Bukit Jalil itu sendiri dijadikan Kolej atau Universiti Sukan Malaysia kerana lengkap dari segi kemudahan prasarana sukannya dan proses pengajaran dan pembelajaran (PdP) dalam bidang Sains Sukan, Kejurulatihan, Pendidikan Jasmani dan setaraf dengannya.

Pengambilan setiap semester pula hanya terhad kepada atlet berstatus kebangsaan dan antarabangsa sahaja supaya hasrat melahirkan lebih ramai atlet bertaraf Olimpik mudah direalisasikan.

Contohnya, bekas atlet lompat bergalah negara, Roslinda Samsu yang juga pemenang pingat perak Sukan Asia Doha 2006 dan Penerima Anugerah Khas Majlis Anugerah Sukan KPT 2012, terpaksa mengambil masa lebih kurang sembilan tahun untuk menamatkan ijazah Sarjana Muda Pendidikan Jasmani di UPM sepanjang 14 tahun terbabit dalam sukan olahraga.

Sepanjang tempoh bergelar atlet kebangsaan dan mahasiswa, beliau juga memenangi pingat Emas Sukan SEA empat siri berturut-turut pada 2005, 2007, 2009 dan 2011.

Begitu juga atlet kebangsaan seperti Leong Mun Yee (UPM); Pandalela Renong (UM); Bryan Nickson Lomas (UM); Cheng Chu Sian (UPM); Marbawi Sulaiman (UiTM) dan Norasheela Khalid (UPM).

Jika disenaraikan, mungkin lebih ramai lagi. Namun, pernah terlintas di fikiran mengapa hanya atlet dari sukan terjun yang dapat memenangi pingat di Sukan Olimpik? Bagaimana dengan atlet lain yang juga layak secara merit? Apakah kekangan atau masalah dihadapi sebagai atlet dan mahasiswa?

Adakah kewujudan universiti sukan akan memberi impak besar kepada kemajuan sukan negara? Jika dirancang dan diatur dengan cekap dan sistematik, ia perkara tidak mustahil dicapai.
"""

In [19]:
encoded = encoder.encode(cleaning(string3)) + [1]

In [21]:
%%time
l = sess.run(logits, feed_dict = {X: [encoded], top_p: 0.8, temperature: 0.1})

CPU times: user 8min 57s, sys: 10.1 s, total: 9min 7s
Wall time: 49.6 s


In [22]:
encoder.decode(l[0].tolist())

'- Sekiranya anda ingin menjadi atlet Olimpik, anda mungkin mahu mengekalkannya. Itulah hasil kajian baru dari Universiti Kebangsaan Malaysia, yang mendapati bahawa pelajar yang sudah bersara, yang sering mengambil pendekatan yang lebih baik untuk menjadi atlet di Sukan Asia. "Dengan demikian, pelajar yang sudah bersara itu memberi kesan positif prestasi dan prestasi bukannya memberi kesan yang sama seperti atlet lain," kata seorang pakar sukan. "Lain kali, pelajar yang sudah bersara, mengambil pendekatan yang lebih baik untuk menjadi atlet di Sukan Asia. " Tetapi ada yang mengatakan bahawa pelajar yang sudah bersara, yang sering mengambil pendekatan yang lebih baik untuk menjadi atlet di Sukan Asia. "Memandu dan memberi tumpuan kepada kejayaan yang dicapai pada tahun 2007," kata seorang pakar sukan. "Lain kali, pelajar yang sudah bersara, mengambil pendekatan yang lebih baik untuk menjadi atlet di Sukan Asia. " Tetapi ada yang mengatakan bahawa pelajar yang sudah bersara, yang sering 

In [23]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'output/model.ckpt')

'output/model.ckpt'

In [24]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'top_p' in n.name
        or 'temperature' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'gradients' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'top_p',
 'temperature',
 'embeddings/weights',
 'encoder/layer_0/attention/self/LayerNorm/gamma',
 'encoder/layer_0/attention/self/query/kernel',
 'encoder/layer_0/attention/self/key/kernel',
 'encoder/layer_0/attention/self/value/kernel',
 'encoder/layer_0/attention/self/Softmax',
 'encoder/layer_0/attention/self/output/dense/kernel',
 'encoder/layer_0/ffn/LayerNorm/gamma',
 'encoder/layer_0/ffn/dense/kernel',
 'encoder/layer_0/ffn/dense/bias',
 'encoder/layer_0/ffn/dense_1/kernel',
 'encoder/layer_0/ffn/dense_1/bias',
 'encoder/layer_1/attention/self/LayerNorm/gamma',
 'encoder/layer_1/attention/self/query/kernel',
 'encoder/layer_1/attention/self/key/kernel',
 'encoder/layer_1/attention/self/value/kernel',
 'encoder/layer_1/attention/self/Softmax',
 'encoder/layer_1/attention/self/output/dense/kernel',
 'encoder/layer_1/ffn/LayerNorm/gamma',
 'encoder/layer_1/ffn/dense/kernel',
 'encoder/layer_1/ffn/dense/bias',
 'encoder/layer_1/ffn/dense_1/kernel',
 'encoder/laye

In [25]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [26]:
freeze_graph('output', strings)

INFO:tensorflow:Restoring parameters from output/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 365 variables.
INFO:tensorflow:Converted 365 variables to const ops.
10764 ops in the final graph.


In [27]:
from tensorflow.tools.graph_transforms import TransformGraph

In [28]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [29]:
pb = 'output/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
        
inputs = ['Placeholder', 'top_p', 'temperature']
transformed_graph_def = TransformGraph(input_graph_def, 
                                       inputs,
                                       ['logits'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [30]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


In [31]:
g = load_graph('output/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
top_p = g.get_tensor_by_name('import/top_p:0')
temperature = g.get_tensor_by_name('import/temperature:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.Session(graph = g)

In [32]:
%%time
l = test_sess.run(logits, feed_dict = {x: [encoded], top_p: 0.0, temperature: 0.0})
encoder.decode([i for i in l[0].tolist() if i > 0])

CPU times: user 4min 42s, sys: 9.96 s, total: 4min 52s
Wall time: 43.4 s


'- Sekiranya anda ingin menjadi atlet Olimpik, anda mungkin mahu mengekalkannya. Itulah hasil kajian baru dari Universiti Kebangsaan Malaysia, yang mendapati bahawa pelajar yang sudah bersara, yang sering mengambil pendekatan yang lebih baik untuk berjaya seperti Sukan SEA dan Sukan Asia. "Dengan kecemerlangan dan prestasi yang diberikan, universiti di bawah tekanan dan kurang akademik," kata seorang pakar sukan. "Lain kali, pelajar cemerlang yang memberi prestasi yang kuat dan prestasi yang pantas diberi latihan. " Dan itu mungkin bermakna pelajar yang sudah bersara tidak akan dapat memperoleh pingat emas dalam tempoh empat tahun lagi. "Dengan kecemerlangan dan prestasi yang diberikan, universiti di bawah tekanan dan kurang akademik," kata seorang pakar sukan. "Lain kali, pelajar cemerlang yang memberi prestasi yang pantas diberi latihan dan prestasi yang pantas diberi latihan. " Tetapi ada yang mengatakan bahawa pelajar yang sudah bersara tidak akan dapat memperoleh pingat dalam temp